<a href="https://colab.research.google.com/github/mraihanramadhan/Machine-Learning/blob/main/Week7_Distributed%20XGBoost%20with%20Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ray

     |████████████████████████████████| 54.7 MB 33 kB/s 
     |████████████████████████████████| 72 kB 406 kB/s 


In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 12.9 MB/s 


In [3]:
pip install sklearn

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 09:22:39,330	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-12 09:22:41,059	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=282) [09:22:41] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=282) [09:22:41] Tree method is automatically selected to be 'approx' for distributed training.
(pid=306) [09:22:41] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 09:22:46,215	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-12 09:22:47,883	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-12 09:23:45,560	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-12 09:23:45,577	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-12 09:23:45,579	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_3c34f_00000,PENDING,,0.0188947,3,0.640818
train_model_3c34f_00001,PENDING,,0.0108721,5,0.768255
train_model_3c34f_00002,PENDING,,0.00764655,5,0.77175
train_model_3c34f_00003,PENDING,,0.00380119,2,0.799756


(pid=396) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=396)   warnings.warn(LEGACY_WARNING)
(pid=396) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=396)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=396) 2021-11-12 09:23:47,523	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=396) 2021-11-12 09:23:49,535	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Result for train_model_3c34f_00000:
  date: 2021-11-12_09-23-50
  done: false
  experiment_id: a2ddcc71dbfc41119833a7a2de2168b8
  hostname: b49503f2e6e9
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 396
  time_since_restore: 2.92329478263855
  time_this_iter_s: 2.92329478263855
  time_total_s: 2.92329478263855
  timestamp: 1636709030
  timesteps_since_restore: 0
  train-error: 0.052724
  train-logloss: 0.677703
  training_iteration: 1
  trial_id: 3c34f_00000
  
Result for train_model_3c34f_00000:
  date: 2021-11-12_09-23-50
  done: true
  experiment_id: a2ddcc71dbfc41119833a7a2de2168b8
  experiment_tag: 0_eta=0.018895,max_depth=3,subsample=0.64082
  hostname: b49503f2e6e9
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 396
  time_since_restore: 2.987830638885498
  time_this_iter_s: 0.005828857421875
  time_total_s: 2.987830638885498
  timestamp: 1636709030
  timesteps_since_restore: 0
  train-error: 0.02812
  train-logloss: 0.558071
  training_iteration: 10
  t

(pid=396) 2021-11-12 09:23:50,452	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 2.99 seconds (0.91 pure XGBoost training time).


...

(pid=710) 2021-11-12 09:24:04,068	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Result for train_model_3c34f_00003:
  date: 2021-11-12_09-24-04
  done: false
  experiment_id: 29bdc8fd416c474cae3fc9226586bf3b
  hostname: b49503f2e6e9
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 710
  time_since_restore: 3.0220823287963867
  time_this_iter_s: 3.0220823287963867
  time_total_s: 3.0220823287963867
  timestamp: 1636709044
  timesteps_since_restore: 0
  train-error: 0.054482
  train-logloss: 0.690063
  training_iteration: 1
  trial_id: 3c34f_00003
  
Result for train_model_3c34f_00003:
  date: 2021-11-12_09-24-04
  done: true
  experiment_id: 29bdc8fd416c474cae3fc9226586bf3b
  experiment_tag: 3_eta=0.0038012,max_depth=2,subsample=0.79976
  hostname: b49503f2e6e9
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 710
  time_since_restore: 3.0782575607299805
  time_this_iter_s: 0.005346536636352539
  time_total_s: 3.0782575607299805
  timestamp: 1636709044
  timesteps_since_restore: 0
  train-error: 0.038664
  train-logloss: 0.663347
  training_iter

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_3c34f_00000,TERMINATED,172.28.0.2:396,0.0188947,3,0.640818,10,2.98783,0.558071,0.02812
train_model_3c34f_00001,TERMINATED,172.28.0.2:500,0.0108721,5,0.768255,10,3.28385,0.607766,0.017575
train_model_3c34f_00002,TERMINATED,172.28.0.2:604,0.00764655,5,0.77175,10,3.18542,0.631307,0.01406
train_model_3c34f_00003,TERMINATED,172.28.0.2:710,0.00380119,2,0.799756,10,3.07826,0.663347,0.038664


(pid=710) 2021-11-12 09:24:04,980	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.08 seconds (0.91 pure XGBoost training time).
2021-11-12 09:24:05,137	INFO tune.py:630 -- Total run time: 19.58 seconds (19.41 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.007646548587768535, 'subsample': 0.7717503098403691, 'max_depth': 5}
